In [1]:
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/conda/lib/

env: LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/conda/lib/


In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git@64f63a7df2a02cfd144592d9aa9c871b59258c55
!pip install gradio

In [2]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE_MODEL = "bigscience/bloomz-7b1-mt"
LORA_WEIGHTS = "./bloomz-doctor"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map = {'': 0},
)
model = PeftModel.from_pretrained(model, LORA_WEIGHTS, device_map={'': 0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('$LD_LIBRARY_PATH')}
  warn(msg)


In [3]:
def make_prompt(instruction):
        return f"""Nếu bạn là bác sĩ, vui lòng trả lời các câu hỏi y tế dựa trên mô tả của bệnh nhân.

### Instruction:
{instruction}

### Response:"""

In [4]:
def evaluate(q):
    input_ids = tokenizer(make_prompt(q), return_tensors="pt")["input_ids"].to('cuda')
    with torch.no_grad():
        gen_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens = 256,
            do_sample=True,
            temperature=0.5,
            top_k=20,
            repetition_penalty=1.2,
            #eos_token_id=0, # for open-end generation.
            #pad_token_id=tokenizer.eos_token_id,
        )
    origin_output = tokenizer.batch_decode(gen_tokens)[0]
    output = origin_output.split("### Response:")[1].strip()
    return output

In [5]:
#@title GUI
import gradio as gr
examples = [
    ["Thưa bác sĩ, tôi bị nôn, buồn nôn và đau bụng. Gần đây tôi phát hiện ra rằng tôi có môn vị hẹp. Tôi nên dùng thuốc gì cho nó?"],
    ["Chào bác sĩ, dạo gần đây tóc tôi bị rụng rất nhanh vài tháng vừa rồi. Tôi nghĩ rằng tôi có thể bị hội chứng rụng tóc. Tôi nên làm gì?"],
    ["Thưa bác sĩ, tôi đã bị giật và co giật đột ngột trong cơ bắp của tôi. Nó trở nên rất khó quản lý chúng hàng ngày?"],
    ["Tôi nghĩ mình bị ngộ độc carbon monoxide. Tôi cảm thấy chóng mặt và buồn nôn."],
    ["Tôi đã gặp vấn đề về trí nhớ và nhầm lẫn gần đây. Tôi nghĩ mình có thể mắc hội chứng Wernicke Korsakoff."],    
]

title = "Bác sĩ Chat"


def inference(instruction):
  return evaluate(instruction)

io = gr.Interface(
  inference,
  inputs=[gr.Textbox(lines=5)],
  outputs=[
    gr.Textbox(lines=5, label="Bác sĩ Chat")
  ],
  title=title,
  examples=examples
)
io.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7a2a601727f271d76f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
